In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sfp_nsdsyn import *
import warnings
import itertools
import matplotlib as mpl

warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2

output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
results_dir = os.path.join(output_dir, 'sfp_model', 'results_2D')
fig_dir = os.path.join(output_dir, 'figures')
lr = 0.0005
max_epoch = 30000
roi_list = ['V1','V2','V3']
vsc='pRFsize'
nsd_subj_list = [utils.sub_number_to_string(sn, 'nsdsyn') for sn in np.arange(1,9)]
broderick_sn_list = [1, 6, 7, 45, 46, 62, 64, 81, 95, 114, 115, 121]
broderick_subj_list = [utils.sub_number_to_string(i, 'broderick') for i in broderick_sn_list]
ARGS_2D = ['sub','lr','eph','roi']
roi_pal = [sns.color_palette('dark', 10)[:][k] for k in  [3,2,0]]
roi_pal.insert(0, (0.3,0.3,0.3))
stim_classes=['annulus', 'pinwheel', 'forward spiral','reverse spiral']

In [2]:
pt_file_list = []
for subj, roi in itertools.product(nsd_subj_list, roi_list):
    file_name = os.path.join(output_dir, 'sfp_model', 'results_2D', 'nsdsyn', 'corrected',
                             f'model-params_lr-{lr}_eph-{max_epoch}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
nsd_params = model.load_all_models(pt_file_list, *ARGS_2D)

nsd_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', 'nsdsyn', 'precision', f'precision-s_dset-nsdsyn_vs-pRFsize.csv'))
nsd_df = pd.merge(nsd_params, nsd_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

In [3]:
pt_file_list = []
for subj, roi in itertools.product(broderick_subj_list, ['V1']):
    file_name = os.path.join(output_dir,'sfp_model', 'results_2D', 'broderick','corrected', f'model-params_lr-{lr}_eph-{max_epoch}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
broderick_params = model.load_all_models(pt_file_list, *ARGS_2D)

broderick_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', 'broderick', 'precision','corrected', f'precision-s_dset-broderick_vs-pRFsize.csv'))
broderick_df = pd.merge(broderick_params, broderick_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

In [4]:
broderick_df['dset_type'] = 'Broderick et al. V1'
nsd_df['dset_type'] = nsd_df['vroinames'].apply(lambda x: f'NSD {x}')
final_params = pd.concat((broderick_df, nsd_df), axis=0)


In [ ]:
PARAMS_2D = ['sigma', 'slope', 'intercept', 'p_1', 'p_2', 'p_3', 'p_4', 'A_1', 'A_2']

for dset_type in final_params['dset_type'].unique()[:1]:
    dset_df = final_params[final_params['dset_type'] == dset_type]
    n_sub = dset_df['sub'].nunique()
    for param in PARAMS_2D[:1]:
        bootstrapped_df = pd.DataFrame({})
        for i in range(100):
            sampled_df = dset_df[[param, 'precision', 'dset_type']].sample(n=n_sub, replace=True)
            weighted_mean = utils.calculate_weighted_mean(sampled_df, [param], 'precision', groupby=['dset_type'])
            bootstrapped_df = pd.concat([bootstrapped_df, weighted_mean], axis=0)
        bootstrap_results.append(bootstrapped_df)



,dset_type,sigma
0,Broderick et al. V1,2.334953
0,Broderick et al. V1,2.367651
0,Broderick et al. V1,2.173182
0,Broderick et al. V1,2.225989
0,Broderick et al. V1,2.256162
...,...,...
0,Broderick et al. V1,2.129940
0,Broderick et al. V1,1.964808
0,Broderick et al. V1,2.299522
0,Broderick et al. V1,2.219866


In [9]:
dset_type = 'NSD V1'
dset_df = final_params[final_params['dset_type'] == dset_type]
w = utils.bootstrap_weighted_mean(dset_df, ['slope'], 'precision', groupby=['dset_type'])

In [18]:
utils.bootstrap_weighted_mean(dset_df, ['slope'], 'precision', groupby=['dset_type'])

,dset_type,slope
0,NSD V1,0.157066
1,NSD V1,0.152428
2,NSD V1,0.164448
3,NSD V1,0.158804
4,NSD V1,0.160317
...,...,...
95,NSD V1,0.150375
96,NSD V1,0.160522
97,NSD V1,0.151310
98,NSD V1,0.124010


In [ ]:
w = utils.bootstrap_weighted_mean(dset_df, ['slope','intercept'], 'precision', n_samples=dset_df['sub'].nunique(), groupby=['dset_type'])
# Compute percentiles for each variable separately and combine into a DataFrame
w.groupby("dset_type")[["slope", "intercept"]].apply(
    lambda df: df.apply(lambda col: np.round(np.percentile(col, [16, 84]),3))
).reset_index().rename(columns={'level_1': 'ci'}).replace({'ci': {0:'low',1:'high'}})


,dset_type,ci,slope,intercept
0,NSD V1,low,0.134,0.139
1,NSD V1,high,0.168,0.203


In [96]:
PARAMS_2D = ['sigma', 'slope', 'intercept', 'p_1', 'p_2', 'p_3', 'p_4', 'A_1', 'A_2']

ci_df = utils.calculate_confidence_interval(final_params, PARAMS_2D, 'precision', n_samples=final_params['sub'].nunique(), groupby=['dset_type'])
ci_df = ci_df.set_index(['dset_type']).transpose().reset_index().rename_axis(None, axis=1).rename(columns={'index': 'param'})
ci_df

,param,Broderick et al. V1,NSD V1,NSD V2,NSD V3
0,sigma,"[2.128, 2.407]","[2.066, 2.362]","[3.299, 4.178]","[4.061, 4.922]"
1,slope,"[0.091, 0.134]","[0.128, 0.172]","[0.125, 0.216]","[0.121, 0.204]"
2,intercept,"[0.311, 0.441]","[0.106, 0.216]","[0.082, 0.334]","[0.086, 0.287]"
3,p_1,"[0.043, 0.099]","[0.028, 0.131]","[0.102, 0.214]","[-0.007, 0.108]"
4,p_2,"[-0.037, -0.021]","[-0.039, -0.009]","[-0.142, -0.065]","[-0.082, 0.004]"
5,p_3,"[0.023, 0.096]","[-0.128, 0.045]","[-0.596, -0.11]","[-0.743, -0.197]"
6,p_4,"[-0.022, 0.016]","[-0.097, -0.036]","[-0.06, 0.138]","[-0.1, 0.155]"
7,A_1,"[0.033, 0.055]","[0.038, 0.07]","[0.028, 0.056]","[0.024, 0.057]"
8,A_2,"[-0.011, 0.0]","[-0.025, -0.013]","[-0.036, -0.011]","[-0.018, -0.004]"


In [84]:
mean_df =utils.calculate_weighted_mean(final_params, PARAMS_2D, 'precision', groupby=['dset_type'])
mean_df = mean_df.set_index(['dset_type']).transpose().reset_index().rename_axis(None, axis=1).rename(columns={'index': 'param'})
mean_df = np.round(mean_df, 3)
mean_df

,param,Broderick et al. V1,NSD V1,NSD V2,NSD V3
0,sigma,2.276,2.199,3.798,4.461
1,slope,0.115,0.150,0.171,0.168
2,intercept,0.372,0.171,0.254,0.202
3,p_1,0.071,0.081,0.162,0.052
4,p_2,-0.028,-0.022,-0.103,-0.047
5,p_3,0.058,-0.037,-0.342,-0.464
6,p_4,-0.005,-0.066,0.039,0.042
7,A_1,0.045,0.058,0.046,0.044
8,A_2,-0.005,-0.019,-0.025,-0.012


In [97]:
# Combine ci_df and mean_df so each cell has mean and CI list, e.g. '2.276, [2.103, 2.44]'
combined = mean_df.copy()
for col in combined.columns[1:]:  # skip 'param'
    for i, param in enumerate(combined['param']):
        # Get mean and CI for this param and dset_type
        mean_val = combined.loc[i, col]
        # ci_df_fmt may have lists as strings, ensure it's a list
        ci_val = ci_df_fmt.loc[i, col]
        # If ci_val is a numpy array, convert to list
        if hasattr(ci_val, 'tolist'):
            ci_val = ci_val.tolist()
        # Format as requested: 'mean, [low, high]'
        combined.loc[i, col] = f"{mean_val}, {ci_val}"

combined



,param,Broderick et al. V1,NSD V1,NSD V2,NSD V3
0,sigma,"2.276, [2.103, 2.44]","2.199, [2.062, 2.343]","3.798, [3.327, 4.3]","4.461, [4.083, 4.946]"
1,slope,"0.115, [0.095, 0.131]","0.15, [0.132, 0.177]","0.171, [0.133, 0.225]","0.168, [0.124, 0.213]"
2,intercept,"0.372, [0.304, 0.434]","0.171, [0.109, 0.205]","0.254, [0.121, 0.356]","0.202, [0.094, 0.284]"
3,p_1,"0.071, [0.042, 0.096]","0.081, [0.041, 0.131]","0.162, [0.102, 0.221]","0.052, [-0.01, 0.111]"
4,p_2,"-0.028, [-0.04, -0.023]","-0.022, [-0.042, -0.011]","-0.103, [-0.149, -0.068]","-0.047, [-0.08, 0.004]"
5,p_3,"0.058, [0.035, 0.091]","-0.037, [-0.099, 0.043]","-0.342, [-0.606, -0.06]","-0.464, [-0.876, -0.124]"
6,p_4,"-0.005, [-0.017, 0.009]","-0.066, [-0.104, -0.037]","0.039, [-0.057, 0.15]","0.042, [-0.099, 0.196]"
7,A_1,"0.045, [0.034, 0.054]","0.058, [0.04, 0.071]","0.046, [0.032, 0.058]","0.044, [0.027, 0.058]"
8,A_2,"-0.005, [-0.011, 0.002]","-0.019, [-0.025, -0.014]","-0.025, [-0.039, -0.012]","-0.012, [-0.019, -0.005]"
